# 라이브러리

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2024-06-06 04:53:04--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /scl/fi/g8l6dlu5a3ebovcdymluq/mecab-0.996-ko-0.9.2.tar.gz?rlkey=xlooryouhr1l2ufqbt93ohfvy&dl=1 [following]
--2024-06-06 04:53:05--  https://www.dropbox.com/scl/fi/g8l6dlu5a3ebovcdymluq/mecab-0.996-ko-0.9.2.tar.gz?rlkey=xlooryouhr1l2ufqbt93ohfvy&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccd99ff88a8b1b1a53d28bc2e23.dl.dropboxusercontent.com/cd/0/inline/CUSZUVtOjAV1FNRFyGaGRGPCe20r4nY9DUQroACHhbDn2TOuavpxzf-6vL7NfMQjRy18JRilGeFf31JyDTt-aTVWBhZkgSS6fDYeR8i6HjQho0wnwGRuX0XxBnhzpdEWMc3aTB0YGz5zGUNQL4f3-xXl/file?dl=1# [following]
--2024-06-06 04:53:05--  https://uccd99ff88a8b1b1a53d28b

In [ ]:
from konlpy.tag import Okt
from collections import defaultdict
import re
import pandas as pd
import numpy as np

In [ ]:
import nltk
nltk.download('sentiwordnet')
nltk.download('wordnet')

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 데이터 읽기

In [ ]:
# df = pd.read_csv('/data/kakao_restaurant_5.csv')
# df = pd.read_csv('/data/naver_restaurant_5.csv')
df = pd.read_csv('/data/blog_restaurant_5.csv')
df

,store_name,text
0,윤스쿡,​안녕하세요 하찌에요 😺이제 곧 여름이 다가오고 있어요벌써부터 낮에는 28~9도를...
1,윤스쿡,누가 광운대 맛집 어디냐고 물으면나는 [ 윤스쿡 ]이라고 답하겠다!​대학생의 소울푸...
2,윤스쿡,​​이름 : 윤스쿡​도로명 주소 : 서울 노원구 광운로 29 2층 윤스쿡​● 영업시...
3,윤스쿡,​​안녕하세요 하찌에요 😺오늘은 광운대 돈까스 맛집하찌의 단골집인 윤스쿡에서신메뉴가...
4,윤스쿡,"광운대 상권 살리기 프로젝트[광운인의 밥상]​스물아홉 번째, 윤스쿡​​​​​윤스쿡..."
...,...,...
336,연어시장 석계본점,​​​지난주 금요일!고양이 쌤이 석계역에연어 맛집으로 유명한 곳이있다고 같이 가자고...
337,연어시장 석계본점,안녕하세요! 블로그 첫 인사글을 적고 나서 드디어 첫 글을 쓰게 되었네요.첫 리뷰글...
338,연어시장 석계본점,​따끈 따끈한 #석계연어시장 솔직후기!​ 석계 연어시장내돈내산 솔직후기위치: 석계역...
339,연어시장 석계본점,석계역에서 광운대역 방향으로 걷다보면 시장주위에 있는 연어시장18년도에 가고싶었는...


In [ ]:
df['store_name'].unique()

array(['윤스쿡', '한상', '무명칼국수', '라체나1907', '연어시장 석계본점'], dtype=object)

# 동사 형용사 나누기 - 3종류

- 동사가 2개이거나 형용사가 2개일 때로 선택!

In [ ]:
# 동사가 2개이거나 형용사가 2개이면 자름
def split_sentence_by_verb2_adjective2(sentences):
  okt = Okt()
  results = []

  for sentence in sentences:
    pos_tags = okt.pos(sentence, norm=True)  # 문장을 형태소 단위로 분리하여 품사 태깅

    split_sentences = []
    current_sentence = []

    verb_count = 0
    adjective_count = 0
    last_pos = 0  # 마지막으로 추가한 단어의 위 치

    for word, pos in pos_tags:
        current_sentence.append(word)
        if pos.startswith('Verb'):
            verb_count += 1
        elif pos.startswith('Adjective'):
            adjective_count += 1

        # 동사(Verb)가 2개 이상 또는 형용사(Adjective)가 2개 이상인 경우 문장을 분할
        if verb_count >= 2 or adjective_count >= 2:
          split_sentences.append(sentence[last_pos:sentence.find(word, last_pos) + len(word)].strip())
          last_pos = sentence.find(word, last_pos) + len(word)
          current_sentence = []
          verb_count = 0
          adjective_count = 0

    if current_sentence:
      split_sentences.append(sentence[last_pos:].strip())

    results.extend(split_sentences)

  return results

In [ ]:
# # 동사 개수와 형용사의 개수가 합쳐서 2개이면 자름
# def split_sentence_by_verb_adjective2(sentences):
#   okt = Okt()
#   results = []

#   for sentence in sentences:
#     pos_tags = okt.pos(sentence, norm=True)

#     split_sentences = []
#     current_sentence = []

#     verb_adjective_count = 0
#     last_pos = 0  # 마지막으로 추가한 단어의 위치

#     for word, pos in pos_tags:
#       current_sentence.append(word)
#       if pos.startswith('Verb') or pos.startswith('Adjective'):
#         verb_adjective_count += 1
#         if verb_adjective_count >= 2:  # 동사 또는 형용사가 두 개 이상인 경우
#             split_sentences.append(sentence[last_pos:sentence.find(word, last_pos) + len(word)].strip())
#             last_pos = sentence.find(word, last_pos) + len(word)
#             current_sentence = []
#             verb_adjective_count = 0

#     if current_sentence:  # 마지막에 남은 부분도 추가
#       split_sentences.append(sentence[last_pos:].strip())

#     results.extend(split_sentences)

#   return results

In [ ]:
# # 동사나 형용사가 있으면 자름
# def split_sentence_by_verb_adjective(sentences):
#   okt = Okt()
#   results = []

#   for sentence in sentences:
#     pos_tags = okt.pos(sentence)
#     split_sentences = []
#     current_sentence = []

#     last_pos = 0

#     for word, pos in pos_tags:
#       if pos.startswith('Verb') or pos.startswith('Adjective'):
#         split_sentences.append(sentence[last_pos:sentence.find(word, last_pos) + len(word)].strip())
#         last_pos = sentence.find(word, last_pos) + len(word)
#         current_sentence = []

#     if current_sentence:  # 마지막에 남은 부분도 추가
#       split_sentences.append(sentence[last_pos:].strip())

#     results.extend(split_sentences)

#   return results

# 문장 분류

In [ ]:
# 문장 분류 키워드
keywords = ["근데", "그리고", "지만", "그러나", "또한", ".", "!", "?", ",", "\n"] # "하고"를 넣는 게 좋을까...?

# 키워드를 기준으로 문장 분리
def split_sentence_by_keywords(review, keywords):

  pattern = '|'.join(map(re.escape, keywords)) # keywords를 합친거

  split_sentence = re.split(pattern, review) # 실제로 리뷰를 키워드로 나눈거! - 여기서는 1개만 한거야

  return split_sentence

# 분리된 문장 정제
def clean_sentences(sentences):

  repetitive_patterns = r'(ㅎㅎ+|ㅋㅋ+|ㅠㅠ+|ㅡㅡ+|ㅜㅜ+|ㅡㅜ+|ㅜㅡ+|ㅜㅠ+|εïз)'

  cleaned_sentences = []

  for sentence in sentences:
    cleaned_sentence = re.sub(repetitive_patterns, '', sentence) # 이모티콘 및 반복 표현 제거

    cleaned_sentence = re.sub(r'[^\w\s가-힣]', '', cleaned_sentence) # 특수 문자 제거 (한글, 숫자, 공백을 제외한 모든 문자 제거)

    cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence).strip() # 연속된 공백을 하나의 공백으로 변환

    if cleaned_sentence and len(cleaned_sentence) > 1: # 공백으로만 구성된 문장이 아니고 길이가 1보다 큰 경우에만 추가
      cleaned_sentences.append(cleaned_sentence)

  return cleaned_sentences

# 전체 리뷰 데이터를 키워드 분류
def split_sentences(reviews):
  sentences = []
  for review in reviews:
    sentences.extend(split_sentence_by_keywords(review, keywords))

  # 분리된 데이터 정제
  sentences = clean_sentences(sentences)

  # 동사와 형용사로 분리하는 부분 -> 여기서 다른 모듈로 바꾸면됨

  sentences = split_sentence_by_verb2_adjective2(sentences) # 동사 2개 이거나 형용사 2개
  # sentences = split_sentence_by_verb_adjective2(sentences) # 동사와 형용사가 합쳐서 2번나올 때
  # sentences = split_sentence_by_verb_adjective(sentences) # 동사가 나오거나 형용사가 나올 때

  return sentences

# 카테고리 지정 및 모듈화

In [ ]:
# 카테고리 키워드
categories = {
    "맛": ["맛", "맛집", "맛있", "맛없", "달콤", "신선", "부드럽", "쫄깃", "매콤", "짜다", "달아", "식사"],
    "서비스": ["친절", "불친절", "응대", "서비스", "매너", "사장님", "직원", "종업원"],
    "가격": ["비싸", "싸다", "가성비", "가격", "금액", "저렴", "값"],
    "분위기": ["분위기", "아늑", "조용", "시끄럽", "깔끔", "청결", "시설", "환경", "인테리어", "뷰"]
}

In [ ]:
# 문장을 키워드로 분류하는 함수
def classify_sentence(sentence, categories):
    okt = Okt()
    tokens = okt.morphs(sentence)  # 문장을 형태소 단위로 분리
    # print(tokens)
    sentence_categories = []
    for category, keywords in categories.items():
        if any(keyword in tokens for keyword in keywords):
            sentence_categories.append(category)
    return sentence_categories

# 리뷰 데이터를 카테고리별로 분류
def categorize_reviews(reviews, categories):
    category_dict = defaultdict(list)
    preprocessed_reviews = split_sentences(reviews)

    for review in preprocessed_reviews:
        sentences = re.split(r'(?<=[.!?])\s+', review)
        # print(sentences)
        for sentence in sentences:
            sentence_categories = classify_sentence(sentence, categories)
            # print(sentence_categories)
            for category in sentence_categories:
                category_dict[category].append(sentence)

    return category_dict

# 분류 시작

In [ ]:
category_5 = pd.DataFrame(columns=["review", "category", "restaurant"])

for restaurant_name in df['store_name'].unique():
  # 해당 음식점의 리뷰 추출
  reviews_df = df[df['store_name'] == restaurant_name]['text']
  reviews = reviews_df.tolist()

  print(f"음식점: {restaurant_name}")
  # 카테고리별로 리뷰 분류
  categorized_reviews = categorize_reviews(reviews, categories)

  rows = []
  for key, values in categorized_reviews.items():
      for value in values:
          rows.append([value, key, restaurant_name])

  # DataFrame 생성
  a = pd.DataFrame(rows, columns=['review', 'category', 'restaurant'])

  # 기존 데이터프레임과 결합
  category_5 = pd.concat([category_5, a], ignore_index=True)

category_5

음식점: 윤스쿡
음식점: 한상
음식점: 무명칼국수
음식점: 라체나1907
음식점: 연어시장 석계본점


,review,category,restaurant
0,광운대맛집 광운대혼밥 광운대돈까스 광운대 맛집 뚝배기 쫄면순두부 학교종이땡땡땡 광운...,맛,윤스쿡
1,하찌에요 광운대 점심 맛집을 찾기 위해 여기저기 다니고,맛,윤스쿡
2,광운대 학생들한테 돈까스 맛집으로이미 유명한,맛,윤스쿡
3,마늘맛도 살짝 느껴지면서,맛,윤스쿡
4,면발도 쫄깃하면서,맛,윤스쿡
...,...,...,...
2051,완전 그냥 술집 분위기더라구요,분위기,연어시장 석계본점
2052,것도 팔구분위기도 엄청 어둑어둑하니,분위기,연어시장 석계본점
2053,앤틱한 분위기의 찻집에 와,분위기,연어시장 석계본점
2054,디자인이나 인테리어가 기분이 좋아지는,분위기,연어시장 석계본점


In [ ]:
category_5.to_csv('/data/blog_category_5 동사가 나오거나 형용사가 나올 때.csv', index=False)